Importamos spacy y el modelo de español - he escogido el pequeño. También importamos el matcher de spacy.
Creo las variables nlp y matcher para luego trabajar con estas herramientas más fácilmente. nlp analizará el texto y
matcher procesará todos los matches en nuestro texto.

In [1]:
import spacy
from spacy.matcher import Matcher

nlp = spacy.load("es_core_news_sm")
matcher = Matcher(nlp.vocab)

In [2]:
def analisis(string):
    """Una función que nos ayudará a saber cómo analiza SpaCy los elementos de una frase"""
    texto = nlp(string)
    for token in texto:
        print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_)

In [3]:
#Algunos títulos de pueba para comprobar el código más adelante.
datos = "La monja enana, La oreja de Van Gogh, Jarabe de palo, Amaral, La casa azul, Los planetas, La quinta estación, \
Los toreros muertos, Hombres G"

Información sobre el matcher y ejemplos de código: https://spacy.io/usage/rule-based-matching#matcher

In [4]:
#Creo el patrón que queremos buscar y lo añado al matcher- POS = Part of Speech
pattern = [{"POS": "DET"}, {"POS": "NOUN"}, {"POS": "ADJ"}]
matcher.add("det_sust_adj", None, pattern) #arguments: match_id, on_match, *patterns
   

En lugar de None, se puede llamar a una función que hayas creado y que se active cuando se produzca un match.
Ej. de spacy:
def on_match(matcher, doc, id, matches):
      print('Matched!', matches)
En la nueva versión de Spacy parece que on_match es opcional pero no en las antiguas.

Ahora hago que spacy analice nuestros nombres de grupos y los paso por el matcher que va a crear una lista 
en nuestra variable matches.

In [5]:

doc = nlp(datos)
matches = matcher(doc)

print(matches)

[(2052584671301345654, 0, 3), (2052584671301345654, 16, 19), (2052584671301345654, 27, 30)]


Como la lista se ve de esta forma tan rara, vamos a crear un for loop que vaya por la lista y que nos 
imprima los datos que nosotros queramos. En este caso se ven el id y el nombre del patrón, el número de los token/s/es(?)
inicial y final del match y su texto

In [6]:
for match_id, start, end in matches:
    string_id = nlp.vocab.strings[match_id]  # Este es el nombre patrón al que responde el match, por si hay más de uno 
    span = doc[start:end]  # El span del match
    print(match_id, string_id, start, end, span.text)


2052584671301345654 det_sust_adj 0 3 La monja enana
2052584671301345654 det_sust_adj 16 19 La casa azul
2052584671301345654 det_sust_adj 27 30 Los toreros muertos


Abrimos el archivo con la colección de nombres de artistas y grupos que cantan en español, lo leemos, lo procesamos y le pasamos el matcher. Guardamos todo esto en variables que nos servirán más adelante.

In [7]:
#Abrimos el archivo de artistas españoles.
archivo = open('artistas_españoles.txt', 'r', encoding = 'UTF-8')
archivo_leido = archivo.read()
doc = nlp(archivo_leido)
resultados = matcher(doc)
archivo.close()

In [8]:
for match_id, start, end in resultados:
    string_id = nlp.vocab.strings[match_id]  # Esto imprime el patrón al que responde el match, por si hay más de uno 
    span = doc[start:end]  # El span del match
    print(match_id, string_id, start, end, span.text)


2052584671301345654 det_sust_adj 1174 1177 las flores azules
2052584671301345654 det_sust_adj 2365 2368 las flores azules


SpaCy no ha identificado los grupos que esperábamos. Veamos cómo analiza este título sacado del documento:

In [9]:
analisis("La Monja Enana")
analisis("La monja enana")
analisis("la monja enana")

La La DET DET__Definite=Def|Gender=Fem|Number=Sing|PronType=Art det
Monja Monja PROPN PROPN___ ROOT
Enana Enana PROPN PROPN___ flat
La La DET DET__Definite=Def|Gender=Fem|Number=Sing|PronType=Art det
monja monje NOUN NOUN__Gender=Fem|Number=Sing ROOT
enana enano ADJ ADJ__Gender=Fem|Number=Sing amod
la lo DET DET__Definite=Def|Gender=Fem|Number=Sing|PronType=Art det
monja monje NOUN NOUN__Gender=Fem|Number=Sing ROOT
enana enano ADJ ADJ__Gender=Fem|Number=Sing amod


SpaCy ha interpretado las mayúsuclas como un nombre propio. Veamos qué ocurre si convertimos el texto a minúsculas:

In [10]:
nombres = archivo_leido.split('\n')
print(nombres[0:5])

lista_minusc = []

for n in nombres:
    minusc = n.capitalize()
    lista_minusc.append(minusc)

print(lista_minusc[0:5])

nombres_minusc = '\n'.join(lista_minusc)
print(nombres_minusc[0:100])

['Shakira', 'Ska-P', 'Enrique Iglesias', 'Gipsy Kings', 'Juanes']
['Shakira', 'Ska-p', 'Enrique iglesias', 'Gipsy kings', 'Juanes']
Shakira
Ska-p
Enrique iglesias
Gipsy kings
Juanes
Manu chao
Paco de lucía
La oreja de van gogh
Mägo 


In [11]:
doc_minusculas = nlp(nombres_minusc)
resultados_minusculas = matcher(doc_minusculas)
for match_id, start, end in resultados_minusculas:
    string_id = nlp.vocab.strings[match_id]  # Esto imprime el patrón al que responde el match, por si hay más de uno 
    span = doc_minusculas[start:end]  # El span del match
    print(match_id, string_id, start, end, span.text)


2052584671301345654 det_sust_adj 237 240 La casa azul
2052584671301345654 det_sust_adj 510 513 La polla records
2052584671301345654 det_sust_adj 710 713 La habitación roja
2052584671301345654 det_sust_adj 788 791 El columpio asesino
2052584671301345654 det_sust_adj 880 883 La cabra mecánica
2052584671301345654 det_sust_adj 1163 1166 La costa brava
2052584671301345654 det_sust_adj 1174 1177 las flores azules
2052584671301345654 det_sust_adj 1480 1483 La gusana ciega
2052584671301345654 det_sust_adj 1704 1707 Los fresones rebeldes
2052584671301345654 det_sust_adj 1752 1755 Los toreros muertos
2052584671301345654 det_sust_adj 2189 2192 La monja enana
2052584671301345654 det_sust_adj 2366 2369 las flores azules


Sigue habiendo problemas: los resultados 1174 1177 las flores azules y 2366 2369 las flores azules son partes de nombres,no el nombre completo. Esto es fácil de identificar pues no empiezan en mayúscula.

Para evitar esto haremos un nuevo patrón que tenga en cuenta el salto de línea inicial y final, así nos aseguramos de que el patrón corresponde al título completo.

In [12]:
new_pattern = [{"TEXT":"\n"}, {"POS": "DET"}, {"POS": "NOUN"}, {"POS": "ADJ"}, {"TEXT": "\n"}]
matcher.add("linea", None, new_pattern) 


In [13]:
matcher.remove("det_sust_adj") # Eliminamos el patrón anterior

In [14]:
nuevos_resultados = matcher(doc_minusculas)

for match_id, start, end in nuevos_resultados:
    string_id = nlp.vocab.strings[match_id]  # Esto imprime el patrón al que responde el match, por si hay más de uno 
    span = doc_minusculas[start:end]  # El span del match
    print(match_id, string_id, start, end, span.text)

16821187256031522237 linea 236 241 
La casa azul

16821187256031522237 linea 509 514 
La polla records

16821187256031522237 linea 709 714 
La habitación roja

16821187256031522237 linea 787 792 
El columpio asesino

16821187256031522237 linea 879 884 
La cabra mecánica

16821187256031522237 linea 1162 1167 
La costa brava

16821187256031522237 linea 1479 1484 
La gusana ciega

16821187256031522237 linea 1703 1708 
Los fresones rebeldes

16821187256031522237 linea 1751 1756 
Los toreros muertos

16821187256031522237 linea 2188 2193 
La monja enana



Algunos títulos siguen presentando problemas que tienen que ver con la interpretación que hace SpaCy de las palabras. 
La aparición de "La costa brava" es discutible. Probablemente sería más correcto que se hubiera mantenido como "La Costa Brava" y que SpaCy lo interpretara como una entidad geográfica. Además ha interpretado "records" en "La polla records" como si fuera un adjetivo, probablemente debido a que no la reconoce como una palabra en inglés. 

In [15]:
analisis("La polla records")

La La DET DET__Definite=Def|Gender=Fem|Number=Sing|PronType=Art det
polla polla NOUN NOUN__Gender=Fem|Number=Sing ROOT
records records ADJ ADJ__Gender=Masc|Number=Sing amod
